In [ ]:
# GPU 확인
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# 환경 설정 및 GPU 확인
import torch
import os

print('🔧 환경 설정')
print('=' * 50)
print(f'PyTorch 버전: {torch.__version__}')
print(f'CUDA 사용 가능: {torch.cuda.is_available()}')

if torch.cuda.is_available():
    print(f'GPU 이름: {torch.cuda.get_device_name(0)}')
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f'GPU 메모리: {gpu_mem:.2f} GB')
else:
    print('⚠️ GPU 사용 불가. 런타임 > 런타임 유형 변경에서 GPU 선택 필요')
print('=' * 50)

In [ ]:
# Ultralytics 설치

!pip install ultralytics -q
print("✅ Ultralystics 설치 완료")

In [ ]:
# 사전 학습 모델 불러오기

import ultralytics
from ultralytics import YOLO

model = YOLO('yolo11m.pt') #사전 학습된 yolo11s 모델을 로드한다.

print("✅ 모델 불러오기 완료")

In [ ]:
BASE_PATH = '/content/kvasir'

os.makedirs(BASE_PATH, exist_ok=True)
%cd {BASE_PATH}

In [ ]:
#!pip install roboflow

!curl -L "https://app.roboflow.com/ds/DiL1PS1DJJ?key=oIr2ozl0nl" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
!ls -la

In [ ]:
from glob import glob
from pathlib import Path

# 각 폴더의 모든 이미지 경로 수집
print("🔍 이미지 파일 수집 중...")

train_img_list = glob('/content/kvasir/train/images/*.jpg')
valid_img_list = glob('/content/kvasir/valid/images/*.jpg')
test_img_list = glob('/content/kvasir/test/images/*.jpg')

print(f"   Train: {len(train_img_list)}개")
print(f"   Valid: {len(valid_img_list)}개")
print(f"   Test: {len(test_img_list)}개")

In [ ]:
model.train(
    # --- 핵심 설정 ---
    data='/content/data.yaml',
    epochs=100,
    patience=50,           #감소
    batch=16,
    imgsz=640,
    workers=8,

    # # --- 하이퍼파라미터 ---
    # optimizer='AdamW',
    lr0=0.001,             #증가 - 학습 속도가 문제가 아니라 판단
    lrf=0.01,
    cos_lr=True,


    # # --- 데이터 증강 대체로 감소 -> overfitting 방지, cls loss 낮추기, 훈련 안정성 ---
    hsv_h=0.015,              #감소
    hsv_s=0.7,                #감소
    hsv_v=0.4,                #감소
    degrees=25,               #감소
    translate=0.15,           #감소
    scale=0.5,                #감소
    mosaic=1.0,
    mixup=0.1,                #감소
    flipud=0.3,
    fliplr=0.5,

    # # --- 손실 함수 최적화 (작은 객체용) ---
    box=7.5,                  #감소 - 아주 정확한 박스 위치를 찾기 x
    cls=0.5,                  #감소 - 높였으나 역효과. precision은 이미 적정 수준
    dfl=1.5,

    # # # --- 정규화 및 최적화 ---
    weight_decay=0.0001,      #감소
    warmup_epochs=10,         #감소
    warmup_momentum=0.8,
    momentum=0.937,
    dropout = 0.0,


    # --- 시스템 및 출력 설정 ---
    device=0,
    project='kvasir_capsule',
    name='yolo11s_v5',
    exist_ok=True,
    pretrained=True,
    amp=True,
    plots=True,
)

In [ ]:
from IPython.display import Image, display
import os

result_dir = '/content/runs/detect/train'

if os.path.exists(f'{result_dir}/results.png'):
    print('📈 학습 곡선:')
    display(Image(filename=f'{result_dir}/results.png', width=1000))

In [ ]:
if os.path.exists(f'{result_dir}/confusion_matrix.png'):
    print('\n📊 Confusion Matrix:')
    display(Image(filename=f'{result_dir}/confusion_matrix.png', width=600))

In [ ]:
model = YOLO(f'{result_dir}/weights/best.pt')
metrics = model.val(split = 'test', data = 'data.yaml', conf = 0.2, iou = 0.6)

In [ ]:
result = model(source='/content/test/images', save=True)
for i in result:
    i.show()

In [ ]:
import cv2

# =====================
# 1. 모델 로드
# =====================
model = YOLO('/content/colab/AICV_study_yolov11/data/runs/detect/train2/weights/best.pt')

input_video = "/content/colab/AICV_study_yolov11/data/video2.mp4"
output_video = "/content/colab/AICV_study_yolov11/data/result_with_counts3.mp4"

# =====================
# 2. 비디오 정보 읽기
# =====================
cap = cv2.VideoCapture(input_video)

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

cap.release()

# =====================
# 3. VideoWriter 설정
# =====================
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

# =====================
# 4. YOLO 추론 + 저장
# =====================
for r in model(input_video, stream=True, conf=0.25):
    img = r.plot()  # BGR 이미지

    class_counts = {}

    if r.boxes is not None and r.boxes.cls is not None:
        classes = r.boxes.cls.cpu().numpy().astype(int)

        for cls_id in classes:
            class_name = model.names[cls_id]
            class_counts[class_name] = class_counts.get(class_name, 0) + 1

    # 클래스별 count 화면에 표시
    y_offset = 30
    for cls_name, count in class_counts.items():
        cv2.putText(
            img,
            f"{cls_name}: {count}",
            (20, y_offset),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,
            (0, 255, 255),
            2
        )
        y_offset += 30

    out.write(img)

# =====================
# 5. 마무리
# =====================
out.release()

print(f"✅ 저장 완료: {output_video}")
